In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import csv
import sklearn
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

def drivingLogs(dataPath, skipHeader=False):
    """
    Returns the lines from a driving log
    """
    lines = []
    with open(dataPath + '/driving_log.csv') as csvFile:
        reader = csv.reader(csvFile)
        if skipHeader:
            next(reader, None)
        for line in reader:
            lines.append(line)
    return lines

def findImages(dataPath):
    """
    Returns `([centerPaths], [leftPath], [rightPath], [measurement])`
    """
    directories = [x[0] for x in os.walk(dataPath)]
    dataDirectories = list(filter(lambda directory: os.path.isfile(directory + '/driving_log.csv'), directories))
    centerTotal, leftTotal, rightTotal, measurementTotal = [], [], [], []
    for directory in dataDirectories:
        lines = drivingLogs(directory, skipHeader=True)
        center, left, right, measurements = [], [], [], []
        for line in lines:
            measurements.append(float(line[3].strip()))
            center.append(directory + '/' + line[0].strip())
            left.append(directory + '/' + line[1].strip())
            right.append(directory + '/' + line[2].strip())
        centerTotal.extend(center)
        leftTotal.extend(left)
        rightTotal.extend(right)
        measurementTotal.extend(measurements)

    return (centerTotal, leftTotal, rightTotal, measurementTotal)

def combineImages(center, left, right, measurement, correction):
    """
    Returns ([imagePaths], [measurements])
    """
    imagePaths = []
    imagePaths.extend(center)
    imagePaths.extend(left)
    imagePaths.extend(right)
    measurements = []
    measurements.extend(measurement)
    measurements.extend([x + correction for x in measurement])
    measurements.extend([x - correction for x in measurement])
    return (imagePaths, measurements)

def generator(samples, batch_size=32):
    """
    Generate images and measurments for training
    """
    num_samples = len(samples)
    while 1:
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for imagePath, measurement in batch_samples:
                originalImage = cv2.imread(imagePath)
                image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
                images.append(image)
                angles.append(measurement)
                # Flipping
                images.append(cv2.flip(image,1))
                angles.append(measurement*-1.0)

            inputs = np.array(images)
            outputs = np.array(angles)
            yield sklearn.utils.shuffle(inputs, outputs)

def createPreProcessingLayers():
    """
    Creates a model with the initial pre-processing layers.
    """
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    return model

def carModel():
    """
    nVidea Autonomous Car model
    """
    model = createPreProcessingLayers()
    model.add(Conv2D(24,5,5, subsample=(2,2), activation='relu'))
    model.add(Conv2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Conv2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Conv2D(64,3,3, activation='relu'))
    model.add(Conv2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

def main():
    return None

if __name__ == '__main__':
    centerPaths, leftPaths, rightPaths, measurements = findImages('data')
    imagePaths, measurements = combineImages(centerPaths, leftPaths, rightPaths, measurements, 0.2)
    print('Total Images: {}'.format( len(imagePaths)))
    
    from sklearn.model_selection import train_test_split
    samples = list(zip(imagePaths, measurements))
    train_samples, validation_samples = train_test_split(samples, test_size=0.2)

    print('Train samples: {}'.format(len(train_samples)))
    print('Validation samples: {}'.format(len(validation_samples)))
    
    train_generator = generator(train_samples, batch_size=32)
    validation_generator = generator(validation_samples, batch_size=32)

    print (train_samples[:10])
    model = carModel()
    
    model.compile(loss='mse', optimizer='adam')
    history_object = model.fit_generator(train_generator, samples_per_epoch= \
                                         len(train_samples), validation_data=validation_generator, \
                                         nb_val_samples=len(validation_samples), nb_epoch=3, verbose=1)

    model.save('model.h5')
    print(history_object.history.keys())
    print('Loss')
    print(history_object.history['loss'])
    print('Validation Loss')
    print(history_object.history['val_loss'])

Using TensorFlow backend.


Total Images: 32406
Train samples: 25924
Validation samples: 6482
[('data/3rd/IMG/center_2017_12_11_10_56_32_478.jpg', 0.0), ('data/2nd/IMG/center_2017_12_11_10_53_47_157.jpg', -0.2513216), ('data/3rd/IMG/right_2017_12_11_10_56_17_211.jpg', -0.5), ('data/3rd/IMG/left_2017_12_11_10_56_40_218.jpg', 0.2), ('data/0th/IMG/center_2016_12_01_13_33_25_853.jpg', 0.0), ('data/0th/IMG/left_2016_12_01_13_37_57_284.jpg', 0.3670138), ('data/0th/IMG/left_2016_12_01_13_36_54_834.jpg', 0.37658230000000004), ('data/0th/IMG/center_2016_12_01_13_42_17_637.jpg', 0.0), ('data/0th/IMG/right_2016_12_01_13_44_39_498.jpg', -0.2), ('data/0th/IMG/right_2016_12_01_13_45_18_976.jpg', -0.28824026)]
Epoch 1/3
25920/25924 [============================>.] - ETA: 0s - loss: 0.0477

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


25984/25924 [==============================] - 375s - loss: 0.0477 - val_loss: 0.0399
Epoch 2/3
25928/25924 [==============================] - 374s - loss: 0.0433 - val_loss: 0.0421
Epoch 3/3
25984/25924 [==============================] - 373s - loss: 0.0391 - val_loss: 0.0359
dict_keys(['val_loss', 'loss'])
Loss
[0.047658101474324914, 0.043256273710987003, 0.039095465713203541]
Validation Loss
[0.039938036763273621, 0.042074805948596736, 0.03587427445888227]
